In [1]:
#Install dependencies
! pip install datasets
! pip install transformers
! pip install -U openai-whisper
! pip install git+https://github.com/openai/whisper.git 
! pip install jiwer
! pip install ipywidgets

  Cloning https://github.com/openai/whisper.git to c:\users\aguil\appdata\local\temp\pip-req-build-9bok7kcb
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
INFO: pip is looking at multiple versions of openai-whisper to determine which version is compatible with other requirements. This could take a while.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\aguil\AppData\Local\Temp\pip-req-build-9bok7kcb'
ERROR: Ignored the following versions that require a different python version: 0.1.1 Requires-Python >=3.9; 0.1.2 Requires-Python >=3.8; 0.2.0 Requires-Python >=3.8; 0.3.0 Requires-Python >=3.8; 0.3.1 Requires-Python >=3.8; 0.3.2 Requires-Python >=3.8; 0.3.3 Requires-Python >=3.8; 0.4.0 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement tiktoken==0.3.3 (from openai-whisper) (from versions: none)
ERROR: No matching distribution found for tiktoken==0.3.3


# Get Dataset
CIEMPIESS_TEST

In [2]:
#Load the dataset
from datasets import load_dataset, load_metric, Audio
import whisper
ds=load_dataset("ciempiess/ciempiess_test", split="test")

#Downsample to 16kHz
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

SyntaxError: invalid syntax (utils.py, line 108)

In [ ]:
ds

: 

In [ ]:
ds[0]["audio"]

: 

In [ ]:
ds.num_rows

: 

#Whisper-large
* De momento no ocupamos los inputs, esta es una manera más sencilla.
* Queda pendiente probar wav2vec2-large-xlsr-53-spanish-ep5-944h: https://huggingface.co/carlosdanielhernandezmena/wav2vec2-large-xlsr-53-spanish-ep5-944h
* Comparar con el word error rate


In [ ]:
large_model = whisper.load_model("large")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = large_model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

: 

#Whisper-medium

###TO DO: 
Usar el dataset en espaniol y probar el modelo.

Buscar una metrica adecuada y una biblioteca para el testing

Ver porque esta dando samples en ingles y no espaniol al usar el dataset de ciempiess

In [ ]:
medium_model = whisper.load_model("medium")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = medium_model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

: 

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="spanish", task="transcribe")


# load dummy dataset and read audio files
ds=load_dataset("ciempiess/ciempiess_test", split="test")
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 
print(input_features)
# generate token ids
predicted_ids = model.generate(input_features)
print(predicted_ids)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
print(transcription)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)


: 

#Whisper-tiny

In [ ]:
tiny_model = whisper.load_model("tiny")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = tiny_model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

: 

#Haciendo unas pruebas con el voice detection y wav2vec


In [ ]:
!pip install pydub


: 

In [ ]:
import ipywidgets as widgets
from IPython import display as disp
from IPython.display import display, Audio, clear_output
from google.colab import output
import base64
from pydub import AudioSegment
import io
import tempfile
import librosa

: 

In [ ]:
def record_audio(seconds=3,
                 sample_rate=44100,
                 normalize_db=0.1):
    """Record audio from the browser in colab using javascript.
    Based on: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
    Args:
      seconds: Number of seconds to record.
      sample_rate: Resample recorded audio to this sample rate.
      normalize_db: Normalize the audio to this many decibels. Set to None to skip
        normalization step.
    Returns:
      An array of the recorded audio at sample_rate.
    """
    # Use Javascript to record audio.
    record_js_code = """
      const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
      const b2text = blob => new Promise(resolve => {
        const reader = new FileReader()
        reader.onloadend = e => resolve(e.srcElement.result)
        reader.readAsDataURL(blob)
      })
      var record = time => new Promise(async resolve => {
        stream = await navigator.mediaDevices.getUserMedia({ audio: true })
        recorder = new MediaRecorder(stream)
        chunks = []
        recorder.ondataavailable = e => chunks.push(e.data)
        recorder.start()
        await sleep(time)
        recorder.onstop = async ()=>{
          blob = new Blob(chunks)
          text = await b2text(blob)
          resolve(text)
        }
        recorder.stop()
      })
      """
    print('Starting recording for {} seconds...'.format(seconds))
    display(disp.Javascript(record_js_code))
    audio_string = output.eval_js('record(%d)' % (seconds * 1000.0))
    print('Finished recording!')
    audio_bytes = base64.b64decode(audio_string.split(',')[1])
    return audio_bytes_to_np(audio_bytes,
                             sample_rate=sample_rate,
                             normalize_db=normalize_db)
    
def audio_bytes_to_np(wav_data,
                      sample_rate=44100,
                      normalize_db=0.1):
    """Convert audio file data (in bytes) into a numpy array.
    Saves to a tempfile and loads with librosa.
    Args:
      wav_data: A byte stream of audio data.
      sample_rate: Resample recorded audio to this sample rate.
      normalize_db: Normalize the audio to this many decibels. Set to None to skip
        normalization step.
    Returns:
      An array of the recorded audio at sample_rate.
    """
    # Parse and normalize the audio.
    audio = AudioSegment.from_file(io.BytesIO(wav_data))
    audio.remove_dc_offset()
    if normalize_db is not None:
        audio.normalize(headroom=normalize_db)
    # Save to tempfile and load with librosa.
    with tempfile.NamedTemporaryFile(suffix='.wav') as temp_wav_file:
        fname = temp_wav_file.name
        audio.export(fname, format='wav')
        audio_np, unused_sr = librosa.load(fname, sr=sample_rate)
    return audio_np

: 

In [ ]:
#@title Record
#@markdown * Set recording time:

SAMPLE_RATE = 44100
record_seconds =   10#@param {type:"number", min:1, max:10, step:1}

def _record_audio(b):
  global audio
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  display(Audio(audio, rate=SAMPLE_RATE))

button = widgets.Button(description="Start recording...")
button.on_click(_record_audio)
display(button)

: 

# Normalización del texto

Necesitamos normalizar las salidas de whisper, ya que la librería nos proporciona las etiquetas de los audios pero estos están normalizados, este es un paso previo para poder aplicar algunas métricas y que los resultados sean más acertados.

In [ ]:
import re
from unicodedata import normalize

def formatText(entryText):
    '''
    Quita las etiquetas xml para extraer el texto, quita caracteres,convierte a minuscula
    y devuelve lista de palabras sin acentos (Como que hace más de una cosa xd)
    '''
    text = re.sub(r'<[^<]+>', "",entryText)
    text = text.lower()
    characters = "#&¿?¡!'-[]()\/''=`,:~}{+|." #-> caracteres innecesarios que se quitan 
    text = ''.join(x for x in text if x not in characters)
    return text    

def deleteAccents(text):
    '''
    Borrar acentos del texto, exceptuando ñ
    #Obtenido de https://ideone.com/YcXaQD (ideone.com)
    '''
    #NFD (Normalization Form Canonical Decomposition) expresion para eliminar diacríticos 
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    #NFC (#Normalization Form Canonical Composition) volver a su forma compuesta
    text = normalize( 'NFC', text)
    return text

def deleteSpaces(entryText):
    return " ".join(entryText.split())

def normalizeText(entryText):
    newText = formatText(entryText)
    return deleteSpaces(newText)

def normalizeTextDiacritics(entryText):
    newText = formatText(entryText)
    return deleteSpaces(deleteAccents(newText))

: 

In [ ]:
normalizeText("No sé, honestamente hice hace mu#cho este código, cuando estaba en RIT & XD! [algo aqui] +, PERO me parece ineficiente")

: 

#WORD ERROR RATE
Se tiene una referencia, en este caso el label del audio y se tiene el texto generado por whisper.

WER = (S+D+I) / N => WER = (S+D+I) / (S+D+C)
​

Dónde:
* S es el número de sustituciones,
* D es el número de eliminaciones,
* I es el número de inserciones,
* C es el número de aciertos,
* N es el número de palabras en la referencia, N = (S + D + C).

A veces tambien se usa el Word accuracy, Auto Speech Recognition. 

Wacc = 1 - WER

In [ ]:
import jiwer

def getWER(ref, hyp):
  error = jiwer.wer(ref, hyp)
  return error

: 

In [ ]:
# El modelo que quieran
# tiny_model, medium_model, large_model
result = medium_model.transcribe(sample)

: 

In [ ]:
print("WER para el primer simple:", getWER((result["text"]), ds[0]["normalized_text"]))
print((result["text"]),"\n", ds[0]["normalized_text"])

: 

In [ ]:
print("WER para el primer texto normalizado:", getWER(normalizeText(result["text"]), ds[0]["normalized_text"]))
print(normalizeText(result["text"]),"\n", ds[0]["normalized_text"])

: 

In [ ]:
print("WER para el primer normalizado sin diacriticos:", getWER(normalizeTextDiacritics(result["text"]), ds[0]["normalized_text"]))
print(normalizeTextDiacritics(result["text"]),"\n", ds[0]["normalized_text"])

: 

#WER para todo el data set de Ciempies con Whisper tiny
* Tiempo aproximado: 25 mins
* Resultado aproximado: 0.2953889115679953

In [ ]:
import numpy as np

def getWERMetric(model, dataset):
  WERList = []
  for i in range(int(dataset.num_rows)):
    sample = dataset[i]["audio"]["path"]
    result = model.transcribe(sample)
    textResult = normalizeText(result["text"])
    label = dataset[i]["normalized_text"]
    if (textResult!="" and label!=""):
      WERList.append(getWER(textResult, label))
  return np.mean(WERList)

: 

In [ ]:
value = getWERMetric(tiny_model,ds)
value

: 

* Tiempo aproximado: 1 h y 6 s

In [ ]:
value = getWERMetric(medium_model,ds)
value

: 

In [ ]:
value = getWERMetric(large_model,ds)
value

: 

#Voice Activity Detector


In [ ]:
pip install malaya-speech

: 

In [ ]:
import malaya_speech
import numpy as np
import librosa
from malaya_speech import Pipeline

: 

In [ ]:
def norm_mel(y, sr):
    mel = librosa.feature.melspectrogram(y, sr = sr, n_mels = 80)
    return np.log10(np.maximum(mel, 1e-10)).T

def plot(y, sr):
    mel = norm_mel(y, sr)
    fig, axs = plt.subplots(2, figsize=(10, 8))
    axs[0].plot(y)
    im = axs[1].imshow(np.rot90(mel), aspect='auto', interpolation='none')
    fig.colorbar(mappable=im, shrink=0.65, orientation='horizontal', ax=axs[1])
    plt.show()

: 

In [ ]:
y, sr = malaya_speech.load('probando5.wav')
len(y) / sr

: 

In [ ]:
import matplotlib.pyplot as plt
import IPython.display as ipd

: 

In [ ]:
ipd.Audio(y, rate = sr)

: 

In [ ]:
plot(y, sr)

: 

In [ ]:
y_ = librosa.effects.trim(y, top_db = 20)[0]
len(y_) / sr

: 

In [ ]:
ipd.Audio(y_, rate = sr)

: 

#probando otro metodo


In [ ]:
vad = malaya_speech.vad.webrtc()


: 

In [ ]:
y_= malaya_speech.resample(y, sr, 16000)
y_ = malaya_speech.astype.float_to_int(y_)
frames = malaya_speech.generator.frames(y, 30, sr)
frames_ = list(malaya_speech.generator.frames(y_, 30, 16000, append_ending_trail = False))
frames_webrtc = [(frames[no], vad(frame)) for no, frame in enumerate(frames_)]

: 

In [ ]:
p = Pipeline()

pipeline_left = (
    p.map(malaya_speech.generator.frames, frame_duration_ms = 30, sample_rate = 16000)
)

pipeline_right = (
    p.map(malaya_speech.resample, old_samplerate = sr, new_samplerate = 16000)
    .map(malaya_speech.astype.float_to_int)
    .map(malaya_speech.generator.frames, frame_duration_ms = 30, sample_rate = 16000,
         append_ending_trail = False)
    .foreach_map(vad)
)

pipeline_left.foreach_zip(pipeline_right).map(malaya_speech.combine.without_silent)

p.visualize()

: 

In [ ]:
results = p(y)
results.keys()

: 

In [ ]:
y_ = malaya_speech.combine.without_silent(frames_webrtc)
y_

: 

In [ ]:
ipd.Audio(y_, rate = sr)


: 